# RisePro Triage
> Config Extractor for PrivateLoader

- toc: true 
- badges: true
- categories: [risepro,stealer,config,triage]

## Overview

According to [FlashPoint](https://flashpoint.io/blog/risepro-stealer-and-pay-per-install-malware-privateloader/https://flashpoint.io/blog/risepro-stealer-and-pay-per-install-malware-privateloader/) 
>“RisePro” is a newly identified stealer written in C++ that appears to possess similar functionality to the stealer malware “Vidar.” RisePro targets potentially sensitive information on infected machines and attempts to exfiltrate it in the form of logs. “RisePro” is a newly identified stealer written in C++ that appears to possess similar functionality to the stealer malware “Vidar.” RisePro targets potentially sensitive information on infected machines and attempts to exfiltrate it in the form of logs. 

RisePRO triggered a false positive for the [Malpedia PrivateLoader yara rule](https://malpedia.caad.fkie.fraunhofer.de/details/win.privateloader) and triggered the UnpacMe PrivateLoader config extractor. This led to correct string extraction from RisePRO even thought it is a separate malware. Are these related?!


## References 
- [“RisePro” Stealer and Pay-Per-Install Malware “PrivateLoader”](https://flashpoint.io/blog/risepro-stealer-and-pay-per-install-malware-privateloader/)
- [PrivateLoader Triage -  Config Extractor for PrivateLoader](https://research.openanalysis.net/privateloader/loader/config/triage/2022/09/08/privateloader.html)
- [privateloader_str_decrypt.py](https://gist.github.com/andretavare5/66ec413cdb4c7c39d35c22d38c7067a8#file-privateloader_str_decrypt-py)
- [Malpedia yara](https://malpedia.caad.fkie.fraunhofer.de/details/win.privateloader)
- [PrivateLoader: the loader of the prevalent ruzki PPI service](https://blog.sekoia.io/privateloader-the-loader-of-the-prevalent-ruzki-ppi-service/)
- [Example string encryption using xorstr](https://github.com/JustasMasiulis/xorstr)

## Sample
- `2cd2f077ca597ad0ef234a357ea71558d5e039da9df9958d0b8bd0efa92e74c9`[UnpacMe](https://www.unpac.me/results/31bbf769-34dc-4d9c-af52-721c61853713?hash=2cd2f077ca597ad0ef234a357ea71558d5e039da9df9958d0b8bd0efa92e74c9#/)

## Analysis

Both PrivateLoader and RisePRO use the same plaintext user agent string `Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36` and request header `"Content-Type: application/x-www-form-urlencoded"` also the string encryption algorithm is the same (xmm registers and xor intrinsic). Funny enough this causes a 100% overlap between the PrivateLoader Yara rule and RisePro.




### Yara Rule

PrivateLoader yara rule matches on both PrivateLoader and RisePRO!

```
rule win_privateloader_w0 {
  meta:
    author =    "andretavare5"
    org =       "BitSight"
    date =      "2022-06-06"
    md5 =       "8f70a0f45532261cb4df2800b141551d"
    reference = "https://tavares.re/blog/2022/06/06/hunting-privateloader-pay-per-install-service"
    license =   "CC BY-NC-SA 4.0"
    
    malpedia_reference = "https://malpedia.caad.fkie.fraunhofer.de/details/win.privateloader"
    malpedia_version = "20220824"
    malpedia_license = "CC BY-NC-SA 4.0"
    malpedia_sharing = "TLP:WHITE"
  strings:
    $code = { 66 0F EF (4?|8?) } // pxor xmm(1/0) - str chunk decryption
    $str =  "Content-Type: application/x-www-form-urlencoded" wide ascii
    $ua1 = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" wide ascii
    $ua2 = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36" wide ascii
                              
  condition:
    uint16(0) == 0x5A4D and // MZ
    $str and
    any of ($ua*) and
    #code > 100
}
```

We need new Yara rules for both PrivateLoader and RisePRO to differentiate the two. 

### C2
The C2 domain is stored in plain text in RisePRO unlike PrivateLoader (where it is encrypted). 
```
194.169.175[.]128
```


## String Decryption
There are encrypted stack strings that are composed of the string data, and an accompanying XOR key. These are loaded onto the stack, then directly XOR decrypted.

### X-Junior IDA Script
[X-Junior](https://github.com/X-Junior) has a script that we can try in IDA to decrypt these strings: [GitHub Repo](https://github.com/X-Junior/Malware-IDAPython-Scripts/tree/main/PivateLoader).



### Andre Tavares Python Script
[andretavare5](https://gist.github.com/andretavare5) has a python script using capstone to decrypt the strings: [Script Gist](https://gist.github.com/andretavare5/66ec413cdb4c7c39d35c22d38c7067a8#file-privateloader_str_decrypt-py).

We have created our own hybrid of the two, which uses capstone for disassembly, but implements the logic from the IDA script...

### XorStr Library
The string decryption looks a lot like this open source library [xorstr](https://github.com/JustasMasiulis/xorstr). The following is an example of the library in use.

```
.text:00411E14 C7 44 24 08 25 7B 87 92                       mov     [esp+60h+var_58], 92877B25h
.text:00411E1C 0F 57 C0                                      xorps   xmm0, xmm0
.text:00411E1F C7 44 24 0C B6 10 A7 1F                       mov     [esp+60h+var_54], 1FA710B6h
.text:00411E27 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E2B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E2F 89 44 24 10                                   mov     dword ptr [esp+60h+var_50], eax
.text:00411E33 89 4C 24 14                                   mov     dword ptr [esp+60h+var_50+4], ecx
.text:00411E37 C7 44 24 08 D1 77 20 5B                       mov     [esp+60h+var_58], 5B2077D1h
.text:00411E3F C7 44 24 0C C5 36 32 7E                       mov     [esp+60h+var_54], 7E3236C5h
.text:00411E47 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E4B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E4F 89 44 24 18                                   mov     dword ptr [esp+60h+var_50+8], eax
.text:00411E53 89 4C 24 1C                                   mov     dword ptr [esp+60h+var_50+0Ch], ecx
.text:00411E57 C7 44 24 08 6D 1E EB FE                       mov     [esp+60h+var_58], 0FEEB1E6Dh
.text:00411E5F C7 44 24 0C D9 3C 87 48                       mov     [esp+60h+var_54], 48873CD9h
.text:00411E67 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E6B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E6F C7 44 24 08 BE 05 4C 3F                       mov     [esp+60h+var_58], 3F4C05BEh
.text:00411E77 89 44 24 40                                   mov     dword ptr [esp+60h+var_20], eax
.text:00411E7B C7 44 24 0C E4 36 32 7E                       mov     [esp+60h+var_54], 7E3236E4h
.text:00411E83 8B 44 24 08                                   mov     eax, [esp+60h+var_58]
.text:00411E87 89 4C 24 44                                   mov     dword ptr [esp+60h+var_20+4], ecx
.text:00411E8B 8B 4C 24 0C                                   mov     ecx, [esp+60h+var_54]
.text:00411E8F 89 44 24 48                                   mov     dword ptr [esp+60h+var_20+8], eax
.text:00411E93 8D 44 24 10                                   lea     eax, [esp+60h+var_50]
.text:00411E97 89 4C 24 4C                                   mov     dword ptr [esp+60h+var_20+0Ch], ecx
.text:00411E9B 8D 50 01                                      lea     edx, [eax+1]
.text:00411E9E 0F 28 4C 24 40                                movaps  xmm1, [esp+60h+var_20]
.text:00411EA3 66 0F EF 4C 24 10                             pxor    xmm1, [esp+60h+var_50]
.text:00411EA9 0F 29 4C 24 10                                movaps  [esp+60h+var_50], xmm1
.text:00411EAE 0F 29 44 24 20                                movaps  [esp+60h+var_40], xmm0
.text:00411EB3 C7 44 24 30 00 00 00 00                       mov     [esp+60h+var_30], 0
.text:00411EBB C7 44 24 34 00 00 00 00                       mov     [esp+60h+var_2C], 0
```

### String Extraction

The string encryption is almost identical to privateloader, we had to adjust for registers that store immediates for multiple strings in a row. Also, there are situations where the stack string DWORDs are not combined in order. Currently we don't handle this, but the solution is to use the `.dis` memory displacement to determine the order of the the DWORDs. 


### Samples

- maybe cheat
  - `92394d5c170060b09ba4ffba450f44d5d4387693a00ee1aba910a818fa387b3192394d5c170060b09ba4ffba450f44d5d4387693a00ee1aba910a818fa387b31`
- metastealer
  - `6cf8bfba1b221effcb1eccec0c91fb0906d0b8996932167f654680cb3ac53aac6cf8bfba1b221effcb1eccec0c91fb0906d0b8996932167f654680cb3ac53aac`
- meduza stealer x64
  - `2ad84bfff7d5257fdeb81b4b52b8e0115f26e8e0cdaa014f9e3084f518aa61492ad84bfff7d5257fdeb81b4b52b8e0115f26e8e0cdaa014f9e3084f518aa6149`
- meduza stealer x32
 - `29cf1ba279615a9f4c31d6441dd7c93f5b8a7d95f735c0daa3cc4dbb799f66d429cf1ba279615a9f4c31d6441dd7c93f5b8a7d95f735c0daa3cc4dbb799f66d4`
- mpress unpacked risepro
  - `16ae203879efe1912bb8b97ceb0f4645abcde27a987e98a171d59f9c1ec3f76416ae203879efe1912bb8b97ceb0f4645abcde27a987e98a171d59f9c1ec3f764`
- privateloader
  - `1aa2d32ab883de5d4097a6d4fe7718a401f68ce95e0d2aea63212dd9051039481aa2d32ab883de5d4097a6d4fe7718a401f68ce95e0d2aea63212dd905103948`
- rise pro
  - `2cd2f077ca597ad0ef234a357ea71558d5e039da9df9958d0b8bd0efa92e74c92cd2f077ca597ad0ef234a357ea71558d5e039da9df9958d0b8bd0efa92e74c9`
  


### Decryption Algorithm
- select the first section in the PE file, assume this is the code
- scan code for final `pxor` instruction and truncate at this instruction to remove extra code from scanning (handle packers with large first sections)
- linear disassemble the full code block - **not efficient**
- traverse assembly until `pxor` instruction is located
- scan backwards until all immediate data is located for the `xmm` registers
- decrypt xmm data, this is the string chunk
- keep running tally of string chunks and combine if no instructions separate them 



In [1]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)


In [42]:
import pefile
import struct
from capstone import *
from capstone.x86 import *
import re
import time


def is_ascii(s):
    return all(c < 128 or c == 0 for c in s)


def xor(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i % len(key)])
    return bytes(out)



def get_reg_data(instructions, reg_name):
    search_count = 0
    search_limit = 2000
    for inst in instructions:
        
        if search_count > search_limit:
            break
        search_count += 1
        
        if  inst.mnemonic == 'mov' and  inst.operands[0].type == X86_OP_REG  and  inst.operands[1].type == X86_OP_IMM:
            if inst.reg_name(inst.operands[0].reg) == reg_name:
                imm_value = inst.operands[1].value.imm
                data_chunk = struct.pack('<I',imm_value)
                return data_chunk
    return None


def get_data(instructions):
    data_chunks = []
    count = 0
    steps = 0
    steps_flag = 0
    flag_reg = 0
    search_count = 0
    search_limit = 400
    
    for inst_ptr in range(0,len(instructions)):
        inst = instructions[inst_ptr]
        steps +=1
        
        if search_count > search_limit:
            break
        search_count += 1
         
        if  inst.mnemonic == 'call':
            break

        if  inst.mnemonic == 'mov' and  inst.operands[0].type == X86_OP_REG  and  inst.operands[1].type == X86_OP_IMM:
            flag_reg = 1
        
        # if  inst.mnemonic == 'mov' and inst.operands[0].type == X86_OP_MEM and inst.operands[1].type == X86_OP_REG:
        #     reg_name = inst.reg_name(inst.operands[1].reg)
        #     #print(f"Scanning for {reg_name} data....")
        #     result = get_reg_data(instructions[inst_ptr:], reg_name)
        #     if result is None:
        #         #print("scanning failed")
        #         break
        #     #print(f"Found reg {reg_name} data {result}")
        #     data_chunks.append(result)
        #     count += 1
        #     steps = 0
        #     steps_flag = 1
         
        if  inst.mnemonic == 'mov' and ( (inst.operands[0].type == X86_OP_MEM and inst.operands[0].value.mem.disp != 0) or inst.operands[0].type == X86_OP_REG ) and inst.operands[1].type == X86_OP_IMM:
            imm_value = inst.operands[1].value.imm
            #print(hex(imm_value))
            if imm_value & 0xff000000 == 0:
                break
            data_chunk = struct.pack('<I',imm_value)
            data_chunks.append(data_chunk)

            count += 1
            steps = 0
            steps_flag = 1

        if steps == 16 and steps_flag:
            break
         #if steps == 6 and steps_flag:  # if you got some garbage string use this instead of the above
            #break
       
    
    enc_data = data_chunks[0:count//2][::-1]
    key = data_chunks[count//2:count][::-1]
    
    if flag_reg :
        enc_data = sum(zip(enc_data[1::2], enc_data[::2]), ())
        key = sum(zip(key[1::2], key[::2]), ())
    return b''.join(enc_data),b''.join(key)



def get_strings_from_inst(instructions):
    # search, build and decrypt strings
    strings = []
    addr = None
    string = ''

    for i, inst in enumerate(instructions):

        if inst.mnemonic == 'pxor':
        # if inst.address == 0x0045C8A0:
            #print(hex(inst.address))
            #try: # possible string decryption found  
            reversed_instruction_list = instructions[:i][::-1]
            encrypted_str, key = get_data(reversed_instruction_list)
            # print(f"str_len: {len(encrypted_str)}, key_len: {len(key)}")
            # print(encrypted_str.hex())
            # print(key.hex())

            if len(encrypted_str) == 0 or len(key) == 0:
                #print(f"Error at {hex(inst.address)} key or data is missing")
                continue

            if len(encrypted_str) != len(key):
                #print(f"Error at {hex(inst.address)} key and data not equal length")
                continue

            out = bytearray(encrypted_str[j] ^ key[j] for j in range(len(key)))

            #print(out)
            out = out.replace(b'\x00',b'')
            if len(out) == 0:
                continue
            #print(out.decode('utf-8'))

            if is_ascii(out):
                strings.append((inst.address,out.decode('utf-8')))
    return strings


SAMPLE_PATH = '/tmp/maybecheat.bin'
filename = SAMPLE_PATH
pe = pefile.PE(filename)
# Assume the first section is code
txt = pe.sections[0]

# TODO: we don't seem to be disassembling the full section?!!
image_base = pe.OPTIONAL_HEADER.ImageBase
section_rva = txt.VirtualAddress
section_offset = txt.PointerToRawData

section_data = txt.get_data()

pxor_egg = rb'\x66\x0F\xEF'
scan_end = section_data.rfind(pxor_egg)

section_data = section_data[:scan_end]



# disassemble .txt section
pe = pefile.PE(filename)
md = Cs(CS_ARCH_X86, CS_MODE_32) 
md.detail = True
md.skipdata = True
addr = 0



## Time starts
t = time.time()

# instructions = []
# for inst in md.disasm(section_data, image_base + section_rva):
#     instructions.append(inst)

# strings = get_strings_from_inst(instructions)

strings = []
for m in re.finditer(pxor_egg, section_data, re.DOTALL):
    scan_end = m.start()
    instructions = []
    for inst in md.disasm(section_data[scan_end-0x400:scan_end], image_base + section_rva + scan_end - 0x400):
        instructions.append(inst)
    strings += get_strings_from_inst(instructions)




# Benchmark 50.47584390640259 metastealer    
print(f"Benchmark {time.time() - t}")
                           
# print(len(strings))
string_dict = {}  
last_string = ''
for s in strings:
    if last_string != s[1]:
        string_dict[s[0]] = s[1]
    last_string = s[1]

for o in string_dict.keys():
    print(f"{hex(o)} {string_dict[o]}")
#enc string
# D2 EA F6 5F
# 9A 2F 1D 5C               
# 3B A0 36 4E                
# F0 81 B8 52                 
 
# key
# B7 92 86 36
# E8 4E 69 35
# 54 CE 69 37
# 95 E0 CA 52   
    
print("done")


Benchmark 2.145430088043213
0x10061eea KeyValuesSystem
0x10061f3a vstdlib.dllKeyValuesSystem
0x1006205b KeyValuesSystem
K?stem =
0x100621cc KeyValuesSystem
K?stem =
0x1006233d KeyValuesSystem
0x10077f1a C:/windows/fonts/segoeui.ttf
0x100784f7 Head
0x1007855d BodyHead
0x100785c0 	Body
0x10078631 ArmsHead


0x10078e20 Automatic stop
0x10078e82 Force accuracy
0x10078eea  accuracy
0x10078f8d Select
0x10078fe1 

ions
0x100790cb Automatic Scope
0x1007917d %.2f
0x100794fe Damage Override
0x100792ac %.2f
0x1007942f %.2f
0x10079629 Select
0x1007966e HitboxesSelect
0x100799e7 Select
0x10079a3b MultipointsSelect
0x10079dd9 Select
0x10079e2d Safe HitboxesSelect
0x10079f01 Safe Points
0x1007a7c8 %.2f
0x1007a909  DelvF]ay
0x1007a993 %.2f
0x1007a9f5 FOVRCS 
0x1007aa82 %.2f
0x1007aae4 SmooRCS th
0x1007a8bc %.2f
0x1007ab7a XRCS 
0x1007ac04 RCS Y
0x1007ae26 Nearest
0x1007aecb Head
0x1007af2b Body
0x1007af93 Pelvis
0x1007aff6 Select
0x1007b03b es
0x1007b12c %.2f
0x1007b1fa Dy

In [23]:

string_start = 0x0045C1D3
test_addr = 0x0045C8A0 
test_rva = test_addr - image_base
test_offset = pe.get_offset_from_rva(test_rva)

test_addr - string_start


1741

In [44]:
def ida_to_dmp_rva(ida_addr, image_base, dump_image_base):
    ida_rva = ida_addr - image_base
    return ida_rva + dump_image_base

In [60]:
from dumpulator import Dumpulator

dp = Dumpulator("/tmp/rise.dmp", quiet=True)

main_addr = 0x00458C9B
main_rva = main_addr - image_base
dump_image_base = dp.modules.main
main_dump_addr = main_rva + dump_image_base


dp.start(dp.regs.rip, end=main_dump_addr)
print("done")




def read_std_string(dp, addr):
    str_out = b''
    str_size = dp.read_long(addr + 16)
    if str_size > 15:
        str_ptr = dp.read_ptr(addr)
        str_data = dp.read(str_ptr, str_size)
        return str_data
    else:
        str_out = dp.read_str(addr)
    return str_out
    

global_str_addr = ida_to_dmp_rva(0x004F06B0, image_base, dp.modules.main)
print(read_std_string(dp, global_str_addr))

global_str_addr = ida_to_dmp_rva(0x004F06E0, image_base, dp.modules.main)
config = read_std_string(dp, global_str_addr)

    
global_str_addr = ida_to_dmp_rva(0x04F07DC, image_base, dp.modules.main)
print(read_std_string(dp, global_str_addr))

commit(0x6b6000[0x1000], PAGE_READWRITE)
commit(0x6b7000[0x2000], PAGE_READWRITE)
commit(0x6b9000[0x2000], PAGE_READWRITE)
done
50500
bytearray(b'RisePro\r\nTelegram: https://t.me/RiseProSUPPORT')


'0xc60000'